<p align = "center" draggable=”false” ><img src="https://user-images.githubusercontent.com/37101144/161836199-fdb0219d-0361-4988-bf26-48b0fad160a3.png" 
     width="200px"
     height="auto"/>
</p>

# Building a Simple Classifier with LLMs


---

Before we delve too deep into the powerful tool that LLMs are, let's take a moment to remember that most of the models in production are still leveraging traditional Supervised Machine Learning Workflows.

In this notebook we'll cover the following topics:

- [ ] Data Preparation
- [ ] Fine-tuning A Model for Text Classification With 🤗

## Task 1: Prepare the Data

The [20 Newsgroups](http://qwone.com/~jason/20Newsgroups/) data set is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups.

Here are some [details](https://scikit-learn.org/stable/datasets/real_world.html#newsgroups-dataset) about the dataset from Scikit Learn!

Let's load the data and get it into a usable format!

In [1]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset = "train") # YOUR CODE HERE
test = fetch_20newsgroups(subset = "test") # YOUR CODE HERE

Have a look around the data! Take note of things like data types, column names, and everything else!

Also take note of how many classes there are in our labels and mark it down in the cell below!

In [2]:
NUM_LABELS = 20 # YOUR CODE HERE

Now that we have our raw data - let's convert that into some `pd.Series` objects using `pandas`!

In [3]:
import pandas as pd

X, y = pd.Series(train["data"]), pd.Series(train["target"]) # YOUR CODE HERE
X_test, y_test = pd.Series(test["data"]), pd.Series(test["target"]) # YOUR CODE HERE

Now let's get the Hugging Face `datasets` (documentation [here](https://huggingface.co/docs/datasets/index)) library so we can convert our data into a more usable format.

In [4]:
!pip install -U -q datasets

We'll want to first convert our separate series objects into a combined `pd.DataFrame` with columns: `text` and `label`, for our Xs and ys respectively. 

After that, it's as easy as loading the `pd.DataFrame` into a `Dataset` object!

Don't forget to refer to the documentation if you're stuck!

In [5]:
from datasets import Dataset

train_df = pd.DataFrame({
    "text" : X, # YOUR CODE HERE
    "label" : y # YOUR CODE HERE
})

test_df = pd.DataFrame({
    "text" : X_test, # YOUR CODE HERE
    "label" : y_test # YOUR CODE HERE
})

train_ds = Dataset.from_pandas(train_df) # YOUR CODE HERE
test_ds = Dataset.from_pandas(test_df) # YOUR CODE HERE

Now we can cast our label columns to `datasets.features.ClassLabel` objects using `class_encode_column`! (documentation [here](https://huggingface.co/docs/datasets/package_reference/main_classes#datasets.Dataset.class_encode_column))

In [6]:
train_ds = train_ds.class_encode_column("label") # YOUR CODE HERE
test_ds = test_ds.class_encode_column("label") # YOUR CODE HERE

Stringifying the column:   0%|          | 0/11314 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/11314 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/7532 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/7532 [00:00<?, ? examples/s]

Let's split our training data into training/validation sets using `datasets` `ds.train_test_split()` method! We're going to hold **10%** of the data for validation.

Be sure to stratify by the labels, and set your seed to 19. 

This will return a `DatasetDict`, which we'll assign to `data_dsd`.


Again, if needed: [Documentation!](https://huggingface.co/docs/datasets/package_reference/main_classes#datasets.Dataset.train_test_split)

In [7]:
data_dsd = train_ds.train_test_split(test_size=0.1, seed=19, stratify_by_column="label") # YOUR CODE HERE

Now we need to do some reworking so our `DatasetDict` has all the required splits.

In [8]:
data_dsd['validation'] = data_dsd['test']
data_dsd['test'] = test_ds

In [9]:
data_dsd

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10182
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7532
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1132
    })
})

## TASK 2: Fine-tuning a pre-trained model via 🤗

In this task, we'll be fine-tuning a simple classifier for our above data using Hugging Face's `transformers` library (documentation [here](https://huggingface.co/docs/transformers/index)) as well as the `accelerate` library. (documentation [here](https://huggingface.co/docs/accelerate/index))

Before we dive in, let's take a pit stop to discuss what fine-tuning is - in broad strokes.

- Fine-tuning is a transfer learning approach where a pre-trained machine learning model is further trained on new data, often to specialize in a certain task. This process can involve training the entire network or only a subset of it, with untrained layers remaining 'frozen'.

- This method is prevalent in Natural Language Processing (NLP) and convolutional neural networks. In the latter, early layers capturing lower-level features are typically frozen, while in NLP, large models like GPT-2 are fine-tuned for specific tasks, improving their performance. However, full fine-tuning can be computationally costly and might lead to overfitting.

- Although fine-tuning is commonly executed through supervised learning, it can also be done using weak supervision or reinforcement learning. For instance, language models like ChatGPT and Sparrow are fine-tuned using reinforcement learning from human feedback.

Okay, so now that we have had a brief overview of what fine-tuning actually is - let's set ourselves up to do some!

In [10]:
!pip install -U -q transformers==4.28.0 accelerate

Now that we have our dependencies - we're going to need to choose a base model! 

While there are a number of great choices (and feel free to improvise here), we're going to be sticking with the classic: 

DistilBERT (uncased)!

- [Paper](https://arxiv.org/pdf/1910.01108.pdf)
- [Model Card](https://huggingface.co/distilbert-base-uncased)

Let's begin by setting up our deisred model check-point, which we'll pull from the Hugging Face Hub!

In [11]:
model_checkpoint = "distilbert-base-uncased"

Next up - we'll want to ensure that our text is converted into a compatible format for use in fine-tuning our model:

Enter, Tokenization!

We'll be leveraging Hugging Face's `AutoTokenizer` to make this part a breeze! (documentation [here](https://huggingface.co/docs/transformers/v4.20.1/en/model_doc/auto#transformers.AutoTokenizer))

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) # YOUR CODE

Those who read the model card/paper will have noticed that DistilBERT has a maximum input length of 512, so we'll need to make sure our inputs never exceed that length. Let's set up a parameter for our maximum sequence length, as well as create a preprocessing function to truncate any potentially long sequences.

In [13]:
MAX_LEN = 256

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=MAX_LEN) # YOUR CODE HERE

Now we can use our work from earlier to `map` our preprocessing function over our dataset! (documentation [here](https://huggingface.co/docs/datasets/package_reference/main_classes#datasets.Dataset.map))

In [14]:
tokenized_text = data_dsd.map(preprocess_function, batched=True) # YOUR CODE HERE

Map:   0%|          | 0/10182 [00:00<?, ? examples/s]

Map:   0%|          | 0/7532 [00:00<?, ? examples/s]

Map:   0%|          | 0/1132 [00:00<?, ? examples/s]

In [15]:
tokenized_text

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10182
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 7532
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1132
    })
})

We'll be training in TensorFlow - so we're going to want to create our data collator with `return_tensors='tf'`, otherwise we just want to set up our `DataCollatorWithPadding` with our tokenizer.

In [16]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer, return_tensors="tf")

Now we can set up our `tf_datasets`!

We'll want to include the `attention_mask`, `input_ids`, and `label` for each set - as well as shuffling the training set.

In [17]:
BATCH_SIZE = 16

tf_train_set = tokenized_text["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"], # YOUR CODE HERE
    shuffle=True, # YOUR CODE HERE
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
)
tf_validation_set = tokenized_text["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids","label"], # YOUR CODE HERE
    shuffle=False, # YOUR CODE HERE
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
    )
tf_test_set = tokenized_text["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids","label"], # YOUR CODE HERE
    shuffle=False, # YOUR CODE HERE
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
    )

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Now we can set some hyper parameters, and our optimizer!

In [18]:
from transformers import create_optimizer

EPOCHS = 3
batches_per_epoch = len(tokenized_text["train"]) // BATCH_SIZE
total_train_steps = int(batches_per_epoch * EPOCHS)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

Next up we can load our DistilBERT model using TFAutoModelForSequenceClassification - remember to set the `num_labels` correctly!

In [19]:
from transformers import TFAutoModelForSequenceClassification

my_bert = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=NUM_LABELS) # YOUR CODE HERE

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

We're going to use the naive example of `accuracy` for this notebook - but feel free to use whatever metric you believe will work best.

In [20]:
my_bert.compile(optimizer=optimizer,  metrics=['accuracy'])

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


It's finally time for `model.fit`, let's go!

In [ ]:
%%time
my_bert.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3) # YOUR CODE HERE

Epoch 1/3
325/637 [==============>...............] - ETA: 49s - loss: 1.8336 - accuracy: 0.5581

Once we've trained it - we can measure its performance on the test set. 

In [ ]:
bert_loss, bert_acc = my_bert.evaluate(tf_test_set) # YOUR CODE HERE

Now that we have a model we're satisfied with - let's push it to the Hugging Face Hub!

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
MODEL_NAME = "distilbert_classifier_newsgroups" # YOUR CODE HERE
HUGGINGFACE_ACCT_NAME = "FourthBrainGenAI" # YOUR CODE HERE

my_bert.push_to_hub(f"{HUGGINGFACE_ACCT_NAME}/{MODEL_NAME}")
tokenizer.push_to_hub(f"{HUGGINGFACE_ACCT_NAME}/{MODEL_NAME}")

You're done! Now your model should be on the hub and you can edit/improve your model card! 🚀